# Black-Friday-Practice-Problem

## Introduction

This is my contribution to the Black-Friday Problem competition at:

https://datahack.analyticsvidhya.com/contest/black-friday/<br>

Further documentation in README.md

In [1]:
import graphlab

## Importing the data

In [4]:
train = graphlab.SFrame('train.csv')
test = graphlab.SFrame('test.csv')

Finished parsing file C:\Users\shekh\Documents\ml-road\projects\analyticsvidhya-blackfriday\train.csv

Parsing completed. Parsed 100 lines in 1.89094 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,str,str,long,str,str,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\shekh\Documents\ml-road\projects\analyticsvidhya-blackfriday\train.csv

Parsing completed. Parsed 550068 lines in 1.28284 secs.

Finished parsing file C:\Users\shekh\Documents\ml-road\projects\analyticsvidhya-blackfriday\test.csv

Parsing completed. Parsed 100 lines in 0.852712 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,str,str,long,str,str,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\shekh\Documents\ml-road\projects\analyticsvidhya-blackfriday\test.csv

Parsing completed. Parsed 233599 lines in 0.581601 secs.

In [9]:
train.head()

User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years ...,Marital_Status,Product_Category_1
1000001,P00069042,F,0-17,10,A,2,0,3
1000001,P00248942,F,0-17,10,A,2,0,1
1000001,P00087842,F,0-17,10,A,2,0,12
1000001,P00085442,F,0-17,10,A,2,0,12
1000002,P00285442,M,55+,16,C,4+,0,8
1000003,P00193542,M,26-35,15,A,3,0,1
1000004,P00184942,M,46-50,7,B,2,1,1
1000004,P00346142,M,46-50,7,B,2,1,1
1000004,P0097242,M,46-50,7,B,2,1,1
1000005,P00274942,M,26-35,20,A,1,1,8


In [6]:
train.column_names()

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase']

In [10]:
test.column_names() # we have to predict 'Purchase' for test data also. But here i'll be using the recomme
                    # nder approach

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3']

In [11]:
# converting Purchase data to float from previous string
train['Purchase']=train['Purchase'].astype(float)

## Setting 3 datasets with similar hyperparameters

In [16]:
train_base=graphlab.SFrame({'user_id':train['User_ID'],'item_id':train['Product_ID'],'Purchase':train['Purchase']})
train_user=graphlab.SFrame({'user_id':train['User_ID'],'Gender':train['Gender'],'Age':train['Age'],'Occupation':train['Occupation'],'City_Category':train['City_Category'],'Stay_In_Current_City_Years':train['Stay_In_Current_City_Years'],'Marital_Status':train[ 'Marital_Status']})
train_product=graphlab.SFrame({'item_id':train['Product_ID'],'Product_Category_1':train['Product_Category_1']})

test_base=graphlab.SFrame({'user_id':test['User_ID'],'item_id':test['Product_ID']})
test_user=graphlab.SFrame({'user_id':test['User_ID'],'Gender':test['Gender'],'Age':test['Age'],'Occupation':test['Occupation'],'City_Category':test['City_Category'],'Stay_In_Current_City_Years':test['Stay_In_Current_City_Years'],'Marital_Status':test[ 'Marital_Status']})
test_product=graphlab.SFrame({'item_id':test['Product_ID'],'Product_Category_1':test['Product_Category_1']})

### Model1

In [15]:
foo_one = graphlab.factorization_recommender.create(train_base, target='Purchase',user_data=train_user, 
                                                    item_data=train_product, num_factors=70,
                                                   side_data_factorization=True,random_seed=50)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 550068 observations with 5891 users and 3631 items.

Data prepared in: 2.15748s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 3.70872e+006

Final training RMSE: 1925.8

In [17]:
predicted = foo_one.predict(test_base, new_user_data=test_user, new_item_data=test_product)

In [20]:
purchase=graphlab.SArray(predicted)
User_ID=graphlab.SArray(test['User_ID'])
Product_ID=graphlab.SArray(test['Product_ID'])
Submission=graphlab.SFrame({'User_ID':User_ID,'Product_ID':Product_ID,'Purchase':purchase})
Submission.save('sub1',format='csv')

### Done